Conversion de imágenes

In [3]:
# instalamos opencv
#pip install opencv-python tqdm

# e importamos las librerías
import os
import cv2
from tqdm import tqdm
import os
import cv2
import matplotlib.pyplot as plt
import random

In [4]:


# 
images_dir = 'VisDrone2019-DET-train/images'     # Carpeta con imágenes .jpg
annotations_dir = 'VisDrone2019-DET-train/annotations'  # Carpeta con anotaciones .txt
output_labels_dir = 'labels_yolo'  # Carpeta de salida para anotaciones en formato YOLO

os.makedirs(output_labels_dir, exist_ok=True)

#  Función para conversión a YOLO format
def convert_to_yolo(box, img_width, img_height):
    x, y, w, h = box
    x_center = (x + w / 2) / img_width
    y_center = (y + h / 2) / img_height
    w /= img_width
    h /= img_height
    return x_center, y_center, w, h

#  Recorremos todos los archivos
for file_name in tqdm(os.listdir(annotations_dir)):
    if not file_name.endswith('.txt'):
        continue

    image_name = file_name.replace('.txt', '.jpg')
    image_path = os.path.join(images_dir, image_name)
    
    # Leemos imagen para obtener dimensiones
    if not os.path.exists(image_path):
        continue  # por si falta alguna imagen
    img = cv2.imread(image_path)
    h_img, w_img, _ = img.shape

    # Leemos anotaciones
    txt_path = os.path.join(annotations_dir, file_name)
    with open(txt_path, 'r') as f:
        lines = f.readlines()

    yolo_annotations = []
    for line in lines:
        parts = line.strip().split(',')
        if len(parts) < 5:
            continue
        
        x, y, w, h = map(int, parts[0:4])
        cls = int(parts[5])  # clase VisDrone

        if cls != 1:  # Filtramos solo personas
            continue

        x_center, y_center, width, height = convert_to_yolo((x, y, w, h), w_img, h_img)
        yolo_annotations.append(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    # Guardamos nuevo archivo .txt con anotaciones YOLO
    if yolo_annotations:
        output_path = os.path.join(output_labels_dir, file_name)
        with open(output_path, 'w') as f_out:
            f_out.write('\n'.join(yolo_annotations))


  0%|          | 1/6471 [00:00<05:54, 18.25it/s]


OSError: [Errno 28] No space left on device

In [ ]:
# Rutas
images_path = 'VisDrone2019-DET-train/images'
labels_path = 'VisDrone2019-DET-train/labels_yolo'

# Tamaño de la imagen para escalar las anotaciones
def draw_boxes(image_path, label_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w, _ = img.shape
    
    with open(label_path, 'r') as f:
        for line in f.readlines():
            class_id, x_center, y_center, bw, bh = map(float, line.strip().split())
            # Conversión YOLO -> esquinas
            x1 = int((x_center - bw / 2) * w)
            y1 = int((y_center - bh / 2) * h)
            x2 = int((x_center + bw / 2) * w)
            y2 = int((y_center + bh / 2) * h)
            # Dibujar caja
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, str(int(class_id)), (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
    return img

# Mostrar una imagen aleatoria
random_file = random.choice(os.listdir(labels_path))
image_file = os.path.join(images_path, random_file.replace('.txt', '.jpg'))
label_file = os.path.join(labels_path, random_file)

image_with_boxes = draw_boxes(image_file, label_file)

plt.figure(figsize=(10, 8))
plt.imshow(image_with_boxes)
plt.axis('off')
plt.title(f"Visualización de: {random_file}")
plt.show()

In [ ]:
Se divide el dataset

In [ ]:
import os
import shutil
import random

# Definir paths de origen
IMAGES_DIR = 'VisDrone2019-DET-train/images'
LABELS_DIR = 'VisDrone2019-DET-train/labels_yolo'

# Definir paths de destino
DEST_DIR = 'VisDrone2019-YOLO'
split_ratio = 0.8  # 80% train, 20% val

# Crear estructura de carpetas
for split in ['train', 'val']:
    os.makedirs(os.path.join(DEST_DIR, 'images', split), exist_ok=True)
    os.makedirs(os.path.join(DEST_DIR, 'labels', split), exist_ok=True)

# Obtener todos los archivos de etiqueta (sin extensión)
label_files = [f for f in os.listdir(LABELS_DIR) if f.endswith('.txt')]
random.shuffle(label_files)

# Dividir en train y val
split_index = int(len(label_files) * split_ratio)
train_files = label_files[:split_index]
val_files = label_files[split_index:]

# Función para copiar pares imagen/etiqueta
def move_files(files, split):
    for file in files:
        label_src = os.path.join(LABELS_DIR, file)
        image_src = os.path.join(IMAGES_DIR, file.replace('.txt', '.jpg'))

        label_dst = os.path.join(DEST_DIR, 'labels', split, file)
        image_dst = os.path.join(DEST_DIR, 'images', split, file.replace('.txt', '.jpg'))

        if os.path.exists(image_src):
            shutil.copy(label_src, label_dst)
            shutil.copy(image_src, image_dst)

# Mover los archivos
move_files(train_files, 'train')
move_files(val_files, 'val')

print(f"✅ Split completado: {len(train_files)} train / {len(val_files)} val")


Entrenamiento Con Yolov8

In [13]:
!pip install ultralytics
!pip install torch

Entrenamiento de modelo YoloV8 - Traspasado a Google Colab - Limitacion Hardware

In [6]:
from ultralytics import YOLO

# Usar una versión más potente del modelo base
model = YOLO('yolov8l.pt')  # O incluso yolov8l.pt si tienes buena GPU

# Entrenar
model.train(
    data='C:/Users/Lorena/Documents/UNIR/2semestre/TFM/data.yaml',
    epochs=80,        # Más épocas suele ayudar
    imgsz=960,         # Aumentar resolución mejora la detección de objetos pequeños
    batch=8,           # Ajusta al máximo que permita tu GPU
    name='visdrone-person-m960',
    device='cpu'
)



100%|██████████| 83.7M/83.7M [00:05<00:00, 16.6MB/s]


New https://pypi.org/project/ultralytics/8.3.108 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.106  Python-3.12.7 torch-2.6.0+cpu 


ValueError: Invalid CUDA 'device=0' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 0
os.environ['CUDA_VISIBLE_DEVICES']: None
See https://pytorch.org/get-started/locally/ for up-to-date torch install instructions if no CUDA devices are seen by torch.


Visualizacion del resultado del entrenamiento que se ha hecho en Google Colab.

In [ ]:
#EN VIVO

from ultralytics import YOLO
import cv2

# Cargar el modelo entrenado
model = YOLO(r"C:\Users\Lorena\Documents\UNIR\2semestre\TFM\VisDrone2019-YOLO\Modelo_Entrenado\best.pt")

# Abrir el video
cap = cv2.VideoCapture(r"C:\Users\Lorena\Documents\UNIR\2semestre\TFM\VisDrone2019-YOLO\Videos_Test\furbo.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Inferencia frame a frame
    results = model(frame)

    # Dibujar las detecciones
    annotated_frame = results[0].plot()

    # Mostrar en vivo
    cv2.imshow("Detecciones en vivo", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 6 persons, 65.4ms
Speed: 3.4ms preprocess, 65.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 89.0ms
Speed: 3.0ms preprocess, 89.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 64.4ms
Speed: 2.6ms preprocess, 64.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 63.7ms
Speed: 4.2ms preprocess, 63.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 67.8ms
Speed: 3.2ms preprocess, 67.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 63.4ms
Speed: 3.3ms preprocess, 63.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 55.0ms
Speed: 3.0ms preprocess, 55.0ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 57.2ms
Speed: 3.4ms preprocess, 57.2ms inference, 0.8ms postprocess per image at shape (

In [4]:
#!pip install ultralytics opencv-python

from ultralytics import YOLO
import cv2
import os

# Cargar el modelo entrenado
model = YOLO(r"C:\Users\Lorena\Documents\UNIR\2semestre\TFM\VisDrone2019-YOLO\Modelo_Entrenado\best.pt")

# Ruta al video de prueba
video_path = r"C:\Users\Lorena\Documents\UNIR\2semestre\TFM\VisDrone2019-YOLO\Videos_Test\everest.mp4"

# Ejecutar inferencia sobre el video y guardar resultado
results = model(video_path, save=True)

# Buscar el video generado en la carpeta de salida
output_dir = "runs/detect/predict"
generated_files = os.listdir(output_dir)
for file in generated_files:
    if file.endswith(".mp4"):
        output_video_path = os.path.join(output_dir, file)
        break

# Reproducir el video resultante
cap = cv2.VideoCapture(output_video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    cv2.imshow("Resultado YOLOv8", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Salir con la tecla 'q'
        break

cap.release()
cv2.destroyAllWindows()




WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/444) C:\Users\Lorena\Documents\UNIR\2semestre\TFM\VisDrone2019-YOLO\Videos_Test\everest.mp4: 384x640 (no detections), 50.5ms
video 1/1 (frame 2/444) C:\Users\Lorena\Documents\UNIR\2semestre\TFM\VisDrone2019-YOLO\Videos_Test\everest.mp4: 384x640 1 person, 46.1ms
video 1/1 (frame 3/444) C:\Users\Lorena\Documents\UNIR\2semestre\TFM\VisDrone2019-YOLO\Videos_Test\everest.mp4: 384x640 1 person, 49.6ms
video 1/1 (frame 4/444) C:\Users\Lorena\

NameError: name 'output_video_path' is not defined

In [34]:
import torch
print("CUDA disponible:", torch.cuda.is_available())
print("Número de GPUs:", torch.cuda.device_count())
print("Nombre de GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No hay GPU")
print(torch.version.cuda)
print(torch.backends.cudnn.version())

CUDA disponible: False
Número de GPUs: 0
Nombre de GPU: No hay GPU
None
None


In [32]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install ultralytics opencv-python



Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.5.1%2Bcu121-cp312-cp312-win_amd64.whl (4.1 MB)
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.5.0%2Bcu121-cp312-cp312-win_amd64.whl (4.1 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.4.1%2Bcu121-cp312-cp312-win_amd64.whl (4.1 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.4.0%2Bcu121-cp312-cp312-win_amd64.whl (4.1 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.3.1%2Bcu121-cp312-cp312-win_amd64.whl (4.1 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.3.0%2Bcu121-cp312-cp312-win_amd64.whl (4.1 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.2.2%2Bcu121-cp312-cp312-win_amd64.whl (4.0 

ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device

